In [1]:
from pathlib import Path
import re
import pandas as pd

In [2]:
text = Path('openelm_3B_out_of_domain_names.txt').read_text()

In [3]:
responses = text.split("\n\n\n")[:-1]

In [4]:
responses = [r.split("\n") for r in responses]

In [5]:
responses = [[re.findall(r"\d+: A: (.+)$", response[0])[0], re.findall(r"\d+: A: (.+)$", response[1])[0]] for response in responses]

In [6]:
responses

[['The answer is l.',
  'The last letter of "Craig" is "g". The last letter of "Guy" is "u". The last letter of "Stacy" is "s". The last letter of "Santana" is "a". Concatenating them is "gus". The answer is gus.'],
 ['The answer is nk.',
  'The last letter of "Gerald" is "d". The last letter of "McKnight" is "a". Concatenating them is "gd". The answer is gd.'],
 ['The answer is l.',
  'The last letter of "LOIS" is "i". The last letter of "JACKSON" is "s". Concatenating them is "is". The answer is is.'],
 ['The answer is l.',
  'The last letter of "DEBORAH" is "h". The last letter of "JONES" is "e". Concatenating them is "he". The answer is he.'],
 ['The answer is l.',
  'The last letter of "MARCUS" is "u". The last letter of "DOMINGUEZ" is "s". Concatenating them is "us". The answer is us.'],
 ['The answer is l.',
  'The last letter of "Marvin" is "v". The last letter of "Sexton" is "n". The last letter of "Dorothy" is "y". The last letter of "Byers" is "s". Concatenating them is "vns

In [7]:
df = pd.DataFrame(responses, columns=['no_cot_answer', 'cot_answer'])

In [8]:
df.head()

,no_cot_answer,cot_answer
0,The answer is l.,"The last letter of ""Craig"" is ""g"". The last le..."
1,The answer is nk.,"The last letter of ""Gerald"" is ""d"". The last l..."
2,The answer is l.,"The last letter of ""LOIS"" is ""i"". The last let..."
3,The answer is l.,"The last letter of ""DEBORAH"" is ""h"". The last ..."
4,The answer is l.,"The last letter of ""MARCUS"" is ""u"". The last l..."


In [9]:
df["no_cot_answer"] = df["no_cot_answer"].str.lower()
df["cot_answer"] = df["cot_answer"].str.lower()

In [10]:
df.to_csv('openelm_out_of_domain_outputs.csv', index=False)

In [11]:
df = pd.read_csv('openelm_out_of_domain_outputs.csv')

In [12]:
df

,no_cot_answer,cot_answer
0,the answer is l.,"the last letter of ""craig"" is ""g"". the last le..."
1,the answer is nk.,"the last letter of ""gerald"" is ""d"". the last l..."
2,the answer is l.,"the last letter of ""lois"" is ""i"". the last let..."
3,the answer is l.,"the last letter of ""deborah"" is ""h"". the last ..."
4,the answer is l.,"the last letter of ""marcus"" is ""u"". the last l..."
...,...,...
495,the answer is lis.,"the last letter of ""lisa"" is ""a"". the last let..."
496,the answer is nt.,"the last letter of ""antonio"" is ""o"". the last ..."
497,the answer is yn.,"the last letter of ""anthony"" is ""y"". the last ..."
498,the answer is rl.,"the last letter of ""roland"" is ""l"". the last l..."


In [13]:
og_data = pd.read_csv('OOD_questions.csv')

In [14]:
og_data = og_data[:len(df)]

In [15]:
df = og_data.join(df)

In [16]:
df["answer"] = df["answer"].str.lower()

In [17]:
df.head()

,question,answer,no_cot_answer,cot_answer
0,Take the last letters of the words in “CRAIG G...,gyya,the answer is l.,"the last letter of ""craig"" is ""g"". the last le..."
1,Take the last letters of the words in “GERALD ...,dtar,the answer is nk.,"the last letter of ""gerald"" is ""d"". the last l..."
2,Take the last letters of the words in “LOIS JA...,snaw,the answer is l.,"the last letter of ""lois"" is ""i"". the last let..."
3,Take the last letters of the words in “DEBORAH...,hsln,the answer is l.,"the last letter of ""deborah"" is ""h"". the last ..."
4,Take the last letters of the words in “MARCUS ...,szen,the answer is l.,"the last letter of ""marcus"" is ""u"". the last l..."


In [18]:
def compute_accuracy(df, no_cot_col, cot_col, answer_col):
    no_cot_answers = df[no_cot_col].to_list()
    cot_answers = df[cot_col].to_list()
    answers = df[answer_col].str.lower()
    answers = answers.to_list()
    answers = zip(no_cot_answers, cot_answers, answers)
    answers = list(answers)
    answers = [[str(a[0]), str(a[1]), str(a[2])] for a in answers]
    answers = [
        [
            re.findall(r"\b(" + answer[2] + r")\b", answer[0]),
            re.findall(r"\b(" + answer[2] + r")\b", answer[1]),
            answer[2],
        ]
        for answer in answers
    ]
    answers = [[len(answer[0]), len(answer[1]), answer[2]] for answer in answers]
    no_cot = 0
    cot = 0
    for a, b, c in answers:
        if a != 0:
            no_cot += 1
        if b != 0:
            cot += 1

    no_cot_percent = no_cot / len(answers) * 100
    cot_percent = cot / len(answers) * 100

    print(f"No COT accuracy: {no_cot_percent}%")
    print(f"COT accuracy: {cot_percent}%")

In [19]:
compute_accuracy(df, 'no_cot_answer', 'cot_answer', 'answer')

No COT accuracy: 0.0%
COT accuracy: 0.2%
